In [4]:
import mujoco
import numpy as np
import mediapy as media

model = mujoco.MjModel.from_xml_path('hemisphere_rat.xml')
data = mujoco.MjData(model)

renderer = mujoco.Renderer(model, width=800, height=600)

cam = mujoco.MjvCamera()
cam.azimuth = 90
cam.elevation = -20
cam.distance = 2.5
cam.lookat[:] = [0, 0, 0]

def get_normal(position):
    return position / np.linalg.norm(position)

def sample_random_direction(normal):
    theta = np.random.uniform(0, 2 * np.pi)
    if np.abs(normal[2]) < 1.0:
        tangent = np.array([-normal[1], normal[0], 0])
    else:
        tangent = np.array([0, -normal[2], normal[1]])
    tangent /= np.linalg.norm(tangent)
    bitangent = np.cross(normal, tangent)
    direction = np.cos(theta) * tangent + np.sin(theta) * bitangent
    return direction


desired_speed = 0.1
timestep = model.opt.timestep
total_steps = 1000 


rat_body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, 'rat')
mass = model.body_mass[rat_body_id]


frames = []

for _ in range(total_steps):
    # current position
    rat_pos = data.xpos[rat_body_id].copy()

    # velocity in world coordinates
    vel = np.zeros(6)
    mujoco.mj_objectVelocity(model, data, mujoco.mjtObj.mjOBJ_BODY, rat_body_id, vel, flg_local=0)
    rat_vel = vel[:3]

    normal = get_normal(rat_pos)
    direction = sample_random_direction(normal)
    desired_velocity = desired_speed * direction
    acceleration = (desired_velocity - rat_vel) / timestep
    force = mass * acceleration
    force -= np.dot(force, normal) * normal
    data.xfrc_applied[rat_body_id, :3] = force


    mujoco.mj_step(model, data)
    data.xfrc_applied[rat_body_id, :] = 0
    renderer.update_scene(data, camera=cam)
    img = renderer.render()

    frames.append(img)

media.show_video(frames, fps=30)


C:\Users\Muhammad Sahal\AppData\Local\Temp\ipykernel_29996\2469933463.py:17: RuntimeWarning: invalid value encountered in divide
  return position / np.linalg.norm(position)


This browser does not support the video tag.
